In [1]:
%pip install -r requirements.txt

/bin/bash: /home/henriquesabino/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import gym
import slimevolleygym
from gym.wrappers.gray_scale_observation import GrayScaleObservation
from gym.wrappers.resize_observation import ResizeObservation
from atari_wrappers import MaxAndSkipEnv, BufferWrapper, ImageToPyTorch
from gym.wrappers.record_video import RecordVideo
import numpy as np

INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SlimeVolleyNoFrameskip-v0'
env = gym.make(env_name)
env = MaxAndSkipEnv(env)
env = ResizeObservation(env, INPUT_SHAPE)
env = GrayScaleObservation(env, True)
env = ImageToPyTorch(env)
env = BufferWrapper(env, WINDOW_LENGTH, np.uint8)

In [5]:
from stable_baselines3 import DQN

model = DQN("CnnPolicy", env, verbose=1, buffer_size=10000)
model.learn(total_timesteps=10, log_interval=4)
model.save("slimeVolley")

del model # remove to demonstrate saving and loading

model = DQN.load("slimeVolley")

env_name = 'SlimeVolleyNoFrameskip-v0'
env = gym.make(env_name)
env = MaxAndSkipEnv(env)
env = ResizeObservation(env, INPUT_SHAPE)
env = GrayScaleObservation(env, True)
env = ImageToPyTorch(env)
env = BufferWrapper(env, WINDOW_LENGTH, np.uint8)

obs = env.reset()

while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    if done:
        break

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


TypeError: len() of unsized object